In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of rows
n = 150_000

# Generate synthetic healthcare dataset
data = pd.DataFrame({
    "PatientID": range(1, n+1),
    "Age": np.random.randint(18, 90, size=n),
    "Gender": np.random.choice(["Male", "Female"], size=n, p=[0.48, 0.52]),
    "BMI": np.round(np.random.normal(27, 5, size=n), 1),  # normal distribution around 27
    "BloodPressure_Systolic": np.random.randint(90, 181, size=n),
    "BloodPressure_Diastolic": np.random.randint(60, 121, size=n),
    "CholesterolLevel": np.random.randint(120, 301, size=n),
    "BloodSugarFasting": np.random.randint(70, 201, size=n),
    "SmokingStatus": np.random.choice(["Never", "Former", "Current"], size=n, p=[0.6, 0.25, 0.15]),
    "ExerciseFreq": np.random.randint(0, 8, size=n),  # 0-7 days per week
    "SleepHours": np.random.randint(4, 11, size=n),   # 4-10 hours per day
    "AlcoholConsumption": np.random.randint(0, 21, size=n),  # units per week
    "MentalHealthScore": np.random.randint(0, 31, size=n),   # 0-30 scale
    "PhysicalHealthScore": np.random.randint(0, 31, size=n), # 0-30 scale
    "HeartDisease": np.random.choice(["Yes", "No"], size=n, p=[0.1, 0.9]),
    "KidneyDisease": np.random.choice(["Yes", "No"], size=n, p=[0.05, 0.95]),
    "CancerHistory": np.random.choice(["Yes", "No"], size=n, p=[0.07, 0.93]),
    "MedicationAdherence": np.random.choice(["Low", "Medium", "High"], size=n, p=[0.2, 0.5, 0.3]),
    "HospitalVisitsLastYear": np.random.poisson(2, size=n),  # avg 2 visits
})

# Derive outcome column based on risk factors
conditions = [
    (data["HeartDisease"] == "Yes") | (data["KidneyDisease"] == "Yes") | (data["CancerHistory"] == "Yes"),
    (data["BMI"] > 30) | (data["BloodPressure_Systolic"] > 140) | (data["CholesterolLevel"] > 240)
]
choices = ["Chronic Illness", "At Risk"]
data["Outcome"] = np.select(conditions, choices, default="Healthy")

# Save to Excel
output_path = "Healthcare_Dataset.xlsx"
data.to_excel(output_path, index=False)

print(f"✅ Healthcare dataset with {n} rows saved as {output_path}")


✅ Healthcare dataset with 150000 rows saved as Healthcare_Dataset.xlsx


In [ ]:
from google.colab import files
files.download("Healthcare_Dataset.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# Load dataset
file_path = "Healthcare_Dataset.xlsx"   # make sure this file is in your working directory
data = pd.read_excel(file_path)

# Split into two groups
group1 = data[data["HeartDisease"] == "Yes"]["BMI"].dropna()
group2 = data[data["HeartDisease"] == "No"]["BMI"].dropna()

# Calculate statistics
mean1, mean2 = group1.mean(), group2.mean()
var1, var2 = group1.var(), group2.var()
n1, n2 = len(group1), len(group2)

# Perform Welch's t-test (unequal variances)
t_stat, p_val_two_tail = stats.ttest_ind(group1, group2, equal_var=False)
p_val_one_tail = p_val_two_tail / 2

# Degrees of freedom (Welch-Satterthwaite equation)
df = (var1/n1 + var2/n2)**2 / ((var1**2)/((n1**2)*(n1-1)) + (var2**2)/((n2**2)*(n2-1)))

# Critical values
alpha = 0.05
t_crit_one_tail = stats.t.ppf(1-alpha, df)
t_crit_two_tail = stats.t.ppf(1-alpha/2, df)

# Print results in table format
print("t-Test: Two-Sample Assuming Unequal Variances\n")
print(f"{'':25} Variable 1 (HeartDisease=Yes)   Variable 2 (HeartDisease=No)")
print(f"Mean{'':20} {mean1:.6f}   {mean2:.6f}")
print(f"Variance{'':16} {var1:.6f}   {var2:.6f}")
print(f"Observations{'':12} {n1}   {n2}")
print(f"Hypothesized Mean Difference  0")
print(f"df{'':24} {int(df)}")
print(f"t Stat{'':20} {t_stat:.6f}")
print(f"P(T<=t) one-tail{'':10} {p_val_one_tail:.6f}")
print(f"t Critical one-tail{'':6} {t_crit_one_tail:.6f}")
print(f"P(T<=t) two-tail{'':10} {p_val_two_tail:.6f}")
print(f"t Critical two-tail{'':6} {t_crit_two_tail:.6f}")


t-Test: Two-Sample Assuming Unequal Variances

                          Variable 1 (HeartDisease=Yes)   Variable 2 (HeartDisease=No)
Mean                     27.008178   26.993774
Variance                 25.261595   25.040171
Observations             15089   134911
Hypothesized Mean Difference  0
df                         18593
t Stat                     0.333998
P(T<=t) one-tail           0.369192
t Critical one-tail       1.644936
P(T<=t) two-tail           0.738385
t Critical two-tail       1.960092


In [ ]:
# Save results to a text file
with open("t_test_results.txt", "w") as f:
    f.write("t-Test: Two-Sample Assuming Unequal Variances\n\n")
    f.write(f"Mean (HeartDisease=Yes): {mean1:.6f}\n")
    f.write(f"Mean (HeartDisease=No): {mean2:.6f}\n")
    f.write(f"Variance (Yes): {var1:.6f}\n")
    f.write(f"Variance (No): {var2:.6f}\n")
    f.write(f"Observations (Yes): {n1}\n")
    f.write(f"Observations (No): {n2}\n")
    f.write(f"df: {int(df)}\n")
    f.write(f"t Stat: {t_stat:.6f}\n")
    f.write(f"P(T<=t) one-tail: {p_val_one_tail:.6f}\n")
    f.write(f"t Critical one-tail: {t_crit_one_tail:.6f}\n")
    f.write(f"P(T<=t) two-tail: {p_val_two_tail:.6f}\n")
    f.write(f"t Critical two-tail: {t_crit_two_tail:.6f}\n")

# Download file
from google.colab import files
files.download("t_test_results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from scipy import stats

# Load your uploaded file
file_path = "Healthcare_Dataset.xlsx"   # ensure it's uploaded in Colab
data = pd.read_excel(file_path)

# Group by SmokingStatus
group_never = data[data["SmokingStatus"] == "Never"]["BMI"].dropna()
group_former = data[data["SmokingStatus"] == "Former"]["BMI"].dropna()
group_current = data[data["SmokingStatus"] == "Current"]["BMI"].dropna()

# Compute descriptive stats
summary = pd.DataFrame({
    "Group": ["Never", "Former", "Current"],
    "Mean BMI": [group_never.mean(), group_former.mean(), group_current.mean()],
    "Variance": [group_never.var(), group_former.var(), group_current.var()],
    "Observations": [len(group_never), len(group_former), len(group_current)]
})

# Perform ANOVA
f_stat, p_val = stats.f_oneway(group_never, group_former, group_current)

print("\nANOVA Data Table (for Healthcare_Dataset.xlsx):\n")
print(summary.to_string(index=False))
print("\nF-statistic:", f_stat)
print("p-value:", p_val)



ANOVA Data Table (for Healthcare_Dataset.xlsx):

  Group  Mean BMI  Variance  Observations
  Never 27.014820 25.011012         90438
 Former 26.952870 24.962092         37350
Current 26.986651 25.437206         22212

F-statistic: 2.062119814407832
p-value: 0.12718768287476048


In [ ]:
# Save ANOVA results to CSV
summary.to_csv("anova_summary.csv", index=False)

# Save test results to TXT
with open("anova_results.txt", "w") as f:
    f.write("ANOVA Test: BMI across SmokingStatus groups\n\n")
    f.write(summary.to_string(index=False))
    f.write("\n\nF-statistic: " + str(f_stat))
    f.write("\nP-value: " + str(p_val))

# Download both files
from google.colab import files
files.download("anova_summary.csv")
files.download("anova_results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from scipy import stats
from google.colab import files

# Step 1: Load full dataset (150,000 rows)
file_path = "Healthcare_Dataset.xlsx"   # upload file in Colab sidebar first
data = pd.read_excel(file_path)

# Step 2: Split BMI values by SmokingStatus groups
group_never = data[data["SmokingStatus"] == "Never"]["BMI"].dropna()
group_former = data[data["SmokingStatus"] == "Former"]["BMI"].dropna()
group_current = data[data["SmokingStatus"] == "Current"]["BMI"].dropna()

# Step 3: Descriptive statistics
summary = pd.DataFrame({
    "Group": ["Never", "Former", "Current"],
    "Mean BMI": [group_never.mean(), group_former.mean(), group_current.mean()],
    "Variance": [group_never.var(), group_former.var(), group_current.var()],
    "Observations": [len(group_never), len(group_former), len(group_current)]
})

# Step 4: Perform ANOVA
f_stat, p_val = stats.f_oneway(group_never, group_former, group_current)

# Step 5: Print results
print("\nANOVA Data Table (All 150,000 rows):\n")
print(summary.to_string(index=False))
print("\nF-statistic:", f_stat)
print("p-value:", p_val)

# Step 6: Save and download results
summary.to_csv("anova_summary.csv", index=False)

with open("anova_results.txt", "w") as f:
    f.write("ANOVA Test: BMI across SmokingStatus groups\n\n")
    f.write(summary.to_string(index=False))
    f.write("\n\nF-statistic: " + str(f_stat))
    f.write("\nP-value: " + str(p_val))

files.download("anova_summary.csv")
files.download("anova_results.txt")



ANOVA Data Table (All 150,000 rows):

  Group  Mean BMI  Variance  Observations
  Never 27.014820 25.011012         90438
 Former 26.952870 24.962092         37350
Current 26.986651 25.437206         22212

F-statistic: 2.062119814407832
p-value: 0.12718768287476048


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load dataset
data = pd.read_excel("Healthcare_Dataset.xlsx")

# The original code attempted to pivot on 'GenHlth', which does not exist.
# I will remove the code that references 'GenHlth'.

# Save for Excel - Saving the original data without the attempted pivot
data.to_excel("ANOVA_Data.xlsx", index=False)

print("ANOVA_Data.xlsx created successfully!")

KeyError: 'GenHlth'

In [ ]:
# ANOVA helper: add GenHlth if missing, prepare ANOVA input, compute ANOVA, save & download files
import pandas as pd
import numpy as np
from scipy import stats
import os

# If using Colab and want automatic download uncomment the next line
IN_COLAB = True

# --- 1) Load your file ---
file_path = "Healthcare_Dataset.xlsx"   # upload this file to Colab or put in notebook folder
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Cannot find '{file_path}' in working directory. Upload the file first.")

df = pd.read_excel(file_path)
print("Loaded file. Columns:", df.columns.tolist())

# --- 2) Ensure BMI exists ---
if 'BMI' not in df.columns:
    raise KeyError("Column 'BMI' not found. ANOVA requires the BMI column.")

# --- 3) Create GenHlth if missing ---
if 'GenHlth' in df.columns:
    print("GenHlth already present. Skipping creation.")
else:
    print("GenHlth not present. Attempting to create it from available columns...")

    # Prefer combining physical & mental measures if available
    phys_col = None
    ment_col = None
    for c in ['PhysicalHealthScore', 'PhysHlth', 'Physical_Health', 'PhysHealth']:
        if c in df.columns:
            phys_col = c
            break
    for c in ['MentalHealthScore', 'MentHlth', 'Mental_Health', 'MentHealth']:
        if c in df.columns:
            ment_col = c
            break

    if phys_col and ment_col:
        print(f"Using '{phys_col}' and '{ment_col}' to derive GenHlth.")
        phys = df[phys_col].fillna(df[phys_col].median())
        ment = df[ment_col].fillna(df[ment_col].median())
        combined = phys + ment   # range typically 0-60 if both 0-30
        # Try quantile binning into 5 roughly-equal groups
        try:
            df['GenHlth'] = pd.qcut(combined, 5, labels=[1,2,3,4,5]).astype(int)
        except Exception:
            # fallback: min-max scale to 1..5
            s = (combined - combined.min())/(combined.max() - combined.min())
            df['GenHlth'] = (np.floor(s*5) + 1).clip(1,5).astype(int)
    else:
        # Fallback: create GenHlth from BMI distribution (approximation)
        print("Physical/Mental columns not found. Creating GenHlth from BMI quantiles (approximation).")
        bmi = df['BMI'].fillna(df['BMI'].median())
        try:
            df['GenHlth'] = pd.qcut(bmi, 5, labels=[1,2,3,4,5]).astype(int)
        except Exception:
            s = (bmi - bmi.min())/(bmi.max() - bmi.min())
            df['GenHlth'] = (np.floor(s*5) + 1).clip(1,5).astype(int)

    print("GenHlth created. Value counts:")
    print(df['GenHlth'].value_counts().sort_index())

# --- 4) Build groups for ANOVA (GenHlth = 1..5) ---
groups = {}
for g in [1,2,3,4,5]:
    grp = df.loc[df['GenHlth'] == g, 'BMI'].dropna().reset_index(drop=True)
    groups[g] = grp
    print(f"Group {g}: n={len(grp)}, mean={grp.mean():.4f}, var={grp.var():.4f}")

# Ensure at least 2 groups have data
non_empty = [g for g in groups if len(groups[g])>0]
if len(non_empty) < 2:
    raise ValueError("Not enough groups with data for ANOVA. Need at least two GenHlth groups with BMI values.")

# --- 5) ANOVA test (one-way) ---
# Only include non-empty groups
data_for_anova = [groups[g] for g in non_empty]
f_stat, p_val = stats.f_oneway(*data_for_anova)
print("\nANOVA (one-way) results:")
print("F-statistic:", f_stat)
print("p-value:", p_val)

# --- 6) Create ANOVA summary table ---
summary = pd.DataFrame({
    "GenHlth": [g for g in non_empty],
    "Mean BMI": [groups[g].mean() for g in non_empty],
    "Variance": [groups[g].var() for g in non_empty],
    "Observations": [len(groups[g]) for g in non_empty]
})

# --- 7) Prepare Excel-style ANOVA input: columns Health 1..5 padded with NaN to equal length ---
max_len = max(len(groups[g]) for g in [1,2,3,4,5])
anova_input = {}
for g in [1,2,3,4,5]:
    s = groups[g]
    # pad with NaN to max_len
    anova_input[f"Health {g}"] = s.reindex(range(max_len)).reset_index(drop=True)

anova_df = pd.DataFrame(anova_input)

# Optionally add a long-format snippet (first rows) with BMI & GenHlth for reference (not required by Excel ANOVA)
long_sample = df[['BMI','GenHlth']].reset_index(drop=True)

# --- 8) Save files ---
out1 = "Healthcare_with_GenHlth.xlsx"
out2 = "ANOVA_Input_Health_Groups.xlsx"
out3 = "anova_summary.csv"
out4 = "anova_results.txt"

df.to_excel(out1, index=False)                # original data plus GenHlth
anova_df.to_excel(out2, index=False)          # Excel-ready ANOVA columns (Health 1..5)
summary.to_csv(out3, index=False)             # summary table
with open(out4, "w") as f:
    f.write("ANOVA Test: BMI across GenHlth groups\n\n")
    f.write(summary.to_string(index=False))
    f.write("\n\nF-statistic: " + str(f_stat))
    f.write("\nP-value: " + str(p_val))

print("\nSaved files:", out1, out2, out3, out4)

# --- 9) (Colab only) trigger downloads ---
if IN_COLAB:
    try:
        from google.colab import files
        files.download(out1)
        files.download(out2)
        files.download(out3)
        files.download(out4)
    except Exception as e:
        print("Automatic downloads failed (this may happen outside Colab). Files are saved in working directory.")


Loaded file. Columns: ['PatientID', 'Age', 'Gender', 'BMI', 'BloodPressure_Systolic', 'BloodPressure_Diastolic', 'CholesterolLevel', 'BloodSugarFasting', 'SmokingStatus', 'ExerciseFreq', 'SleepHours', 'AlcoholConsumption', 'MentalHealthScore', 'PhysicalHealthScore', 'HeartDisease', 'KidneyDisease', 'CancerHistory', 'MedicationAdherence', 'HospitalVisitsLastYear', 'Outcome']
GenHlth not present. Attempting to create it from available columns...
Using 'PhysicalHealthScore' and 'MentalHealthScore' to derive GenHlth.
GenHlth created. Value counts:
GenHlth
1    32956
2    30490
3    27489
4    29365
5    29700
Name: count, dtype: int64
Group 1: n=32956, mean=27.0206, var=25.4059
Group 2: n=30490, mean=26.9987, var=24.9345
Group 3: n=27489, mean=27.0122, var=24.7813
Group 4: n=29365, mean=26.9608, var=25.1696
Group 5: n=29700, mean=26.9818, var=24.9672

ANOVA (one-way) results:
F-statistic: 0.6945053783389419
p-value: 0.5956337636951834

Saved files: Healthcare_with_GenHlth.xlsx ANOVA_Input_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download(out1)  # Healthcare_with_GenHlth.xlsx
files.download(out2)  # ANOVA_Input_Health_Groups.xlsx
files.download(out3)  # anova_summary.csv
files.download(out4)  # anova_results.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Load your dataset
file_path = "Healthcare_Dataset.xlsx"   # Upload this file in Colab or place in your working dir
df = pd.read_excel(file_path)

# If GenHlth does not exist, create it from BMI quantiles
if "GenHlth" not in df.columns:
    df["GenHlth"] = pd.qcut(df["BMI"], 5, labels=[1,2,3,4,5]).astype(int)

# --- Summary table like Excel ---
summary = df.groupby("GenHlth")["BMI"].agg(["count","sum","mean","var"]).reset_index()
summary.columns = ["Groups", "Count", "Sum", "Average", "Variance"]

# --- ANOVA calculations ---
groups = [df[df["GenHlth"] == g]["BMI"].dropna() for g in sorted(df["GenHlth"].unique())]
f_stat, p_val = stats.f_oneway(*groups)

# Between-group & within-group sums of squares
grand_mean = df["BMI"].mean()
ss_between = sum(len(g)*(g.mean()-grand_mean)**2 for g in groups)
ss_within = sum(((g - g.mean())**2).sum() for g in groups)
ss_total = ss_between + ss_within

df_between = len(groups) - 1
df_within = len(df) - len(groups)
df_total = len(df) - 1

ms_between = ss_between/df_between
ms_within = ss_within/df_within

f_crit = stats.f.ppf(1-0.05, df_between, df_within)

# --- Print like Excel ---
print("\nSUMMARY")
print(summary.to_string(index=False))

print("\nANOVA")
anova_table = pd.DataFrame({
    "Source of Variation": ["Between Groups","Within Groups","Total"],
    "SS": [ss_between, ss_within, ss_total],
    "df": [df_between, df_within, df_total],
    "MS": [ms_between, ms_within, ""],
    "F": [f_stat, "", ""],
    "P-value": [p_val, "", ""],
    "F crit": [f_crit, "", ""]
})
print(anova_table.to_string(index=False))



SUMMARY
 Groups  Count       Sum   Average  Variance
      1  30462  610478.1 20.040644  5.678960
      2  29964  730212.8 24.369670  0.695145
      3  29584  798973.0 27.006929  0.520216
      4  30311  899462.2 29.674448  0.729277
      5  29679 1010157.4 34.036100  5.365355

ANOVA
Source of Variation           SS     df             MS              F P-value    F crit
     Between Groups 3.368777e+06      4  842194.210027  323460.221155     0.0  2.371991
      Within Groups 3.905424e+05 149995       2.603703                                 
              Total 3.759319e+06 149999                                                


In [ ]:
files.download("anova_group_summary.csv")
files.download("anova_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

# Load your dataset
df = pd.read_excel("Healthcare_Dataset.xlsx")

# ✅ Step 1: Make sure GenHlth exists
# If missing, create from BMI quantiles
if "GenHlth" not in df.columns:
    df["GenHlth"] = pd.qcut(df["BMI"], 5, labels=[1,2,3,4,5]).astype(int)

# ✅ Step 2: Split BMI values into 5 groups
groups = {}
for g in [1,2,3,4,5]:
    groups[g] = df.loc[df["GenHlth"] == g, "BMI"].reset_index(drop=True)

# ✅ Step 3: Make all columns same length (pad with NaN)
max_len = max(len(groups[g]) for g in groups)
anova_input = {}
for g in groups:
    anova_input[f"Health {g}"] = groups[g].reindex(range(max_len))

anova_df = pd.DataFrame(anova_input)

# ✅ Step 4: Save dataset (full 150,000 rows spread across 5 groups)
anova_df.to_excel("ANOVA_Input_150000.xlsx", index=False)

# ✅ Step 5: Download
files.download("ANOVA_Input_150000.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

# Load your uploaded dataset
df = pd.read_excel("Healthcare_Dataset.xlsx")

# ✅ Ensure GenHlth exists
if "GenHlth" not in df.columns:
    # Create GenHlth from BMI quantiles (5 groups)
    df["GenHlth"] = pd.qcut(df["BMI"], 5, labels=[1,2,3,4,5]).astype(int)

# ✅ Split BMI values into 5 groups
groups = {}
for g in [1,2,3,4,5]:
    groups[g] = df.loc[df["GenHlth"] == g, "BMI"].reset_index(drop=True)

# ✅ Pad all groups to equal length
max_len = max(len(groups[g]) for g in groups)
anova_input = {}
for g in groups:
    anova_input[f"Health {g}"] = groups[g].reindex(range(max_len))

anova_df = pd.DataFrame(anova_input)

# ✅ Save as Excel
anova_df.to_excel("ANOVA_Input_150000.xlsx", index=False)

# ✅ Download
files.download("ANOVA_Input_150000.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>